In [9]:
## protein id transfer to geneid
P2G_file = "/public/ref/UniProt/HUMAN_9606_id.GeneName"
P2Symbol = pd.read_csv(P2G_file,sep='\t',header=None)
P2Symbol.columns = ["Protein","Genecard","GeneSymbol"]
P2Symbol_dict = P2Symbol.set_index(['Protein'])['GeneSymbol'].to_dict()

In [10]:
## 读取各个R-HSA-id对应的属性信息
# Protein Reaction
import pandas as pd
protein_reaction_file= '/public/ref/Msigdb/Reactome/useInfo/Human.ProteinRoleReaction.txt'
prf = pd.read_csv(protein_reaction_file,sep='\t',header=None)
print (prf.loc[0:3,0:3])
prf.columns = ["Protein",'Realation','Reaction']
prf["Gene"] = prf['Protein'].apply(lambda x : P2Symbol_dict[x] if x in P2Symbol_dict.keys() else "None") ## 寻找对应基因名，若没有则为None
prf["Name"] = 'ProteinReaction'
ProteinReaction = prf.set_index(['Reaction'])['Gene'].to_dict()
reaction_gene_dict = {}
for i in range(prf.shape[0]):
    if (prf.loc[i,"Gene"] != "None"):
        reaction_gene_dict.setdefault(prf.loc[i,"Reaction"],[]).append(prf.loc[i,"Gene"]) # 只保留能匹配上基因名的
print (ProteinReaction["R-HSA-1112666"])
print (reaction_gene_dict['R-HSA-1112666'])



            0                 1              2
0  A0A075B6P5  catalystActivity  R-HSA-1112666
1  A0A075B6P5  catalystActivity   R-HSA-166753
2  A0A075B6P5  catalystActivity   R-HSA-166792
3  A0A075B6P5            output   R-HSA-173626
SH3KBP1
['IGKV2-28', 'IGKV2D-30', 'IGKV3D-20', 'IGKV1-12', 'IGLC7', 'IGKV2-29', 'IGKV1D-33', 'IGKV1-33', 'IGKV1-39', 'IGKV1-17', 'IGKV1D-16', 'IGKV1-5', 'IGKV1D-12', 'IGKV2D-40', 'IGKV2D-28', 'IGKV3-20', 'IGKV3-15', 'IGLV1-44', 'IGLV1-47', 'IGLV1-51', 'IGLV1-40', 'IGLV2-14', 'IGLV2-23', 'IGLV2-11', 'IGLV2-8', 'IGLV3-19', 'IGLV3-1', 'IGLV3-25', 'IGLV3-27', 'IGLV6-57', 'IGHV1-69', 'IGHV1-46', 'IGHV3-11', 'IGHV3-48', 'IGHV3-23', 'IGHV3-13', 'IGHV3-53', 'IGHV3-30', 'IGHV3-33', 'IGHV3-7', 'IGHV3-9', 'IGHV2-70', 'IGHV2-5', 'IGHV4-39', 'IGHV4-59', 'IGKC', 'IGHM', 'IGHD', 'IGLV7-43', 'IGKV1-16', 'IGKV1D-39', 'IGKV3-11', 'IGKV2-30', 'IGKV4-1', 'IGKV5-2', 'IGHV4-34', 'IGLC6', 'IGLC1', 'IGLC2', 'IGLC3', 'CD79A', 'VAV1', 'NCK1', 'PLCG2', 'IGHV1-2', 'CD79B', 'SYK', 'G

In [16]:
## 输出ProteinReaction 包含的蛋白对应的基因
fnodes =  open("reactome_reaction.ProteinReactionNodes.txt","w")
for keys,items in reaction_gene_dict.items():
    uniq_gene = ','.join(list(set(reaction_gene_dict[keys])))
    fnodes.write(keys + '\t'+ uniq_gene +'\n')
fnodes.close()

In [7]:
# Complex 
complex_file = "/public/ref/Msigdb/Reactome/Complex_2_Pathway_human.txt"
cf = pd.read_csv(complex_file,sep='\t',header=0,)
print (cf.iloc[0:3,0:3])
cf["Name"] = "Complex"
ComplexReaction = cf.set_index(['complex'])['Name'].to_dict()

         complex        pathway top_level_pathway
0  R-ALL-1006146   R-HSA-977606      R-HSA-168256
1   R-ALL-113656   R-HSA-113501     R-HSA-1640170
2  R-ALL-2239507  R-HSA-3000480     R-HSA-5653656


In [6]:
# Ewas
Ewas_file = "/public/ref/Msigdb/Reactome/Ewas2Pathway_human.txt"
Ewas = pd.read_csv(Ewas_file,sep='\t',header=0)
print (Ewas.iloc[0:3,0:3])
Ewas["Name"] = "Ewas"
EwasReaction = Ewas.set_index(['ewas'])['Name'].to_dict()

            ewas        pathway top_level_pathway
0  R-BAN-5205700  R-HSA-5210891     R-HSA-1643685
1  R-BAN-5205702  R-HSA-5210891     R-HSA-1643685
2  R-BAN-5205707  R-HSA-5210891     R-HSA-1643685


In [11]:
# PPI file
PPI_file = "/public/ref/Msigdb/Reactome/reactome.homo_sapiens.interactions.tab-delimited.txt"
PPI = pd.read_csv(PPI_file,sep='\t',header=0)
print (PPI.iloc[0:3,0:7])
PPI_dict = {}
for index,row in PPI.iterrows():
    gene1 = row[0].strip("uniprotkb:")
    gene2 = row[3].strip("uniprotkb:")
    reaction_id = row[7].strip("reactome:")
    if (gene1 in P2Symbol_dict.keys()):
        PPI_dict.setdefault(reaction_id,[]).append(P2Symbol_dict[gene1])
    if (gene2 in P2Symbol_dict.keys()):
        PPI_dict.setdefault(reaction_id,[]).append(P2Symbol_dict[gene2])
    #break
#print (PPI_dict.values())

  Interactor 1 uniprot id                       Interactor 1 Ensembl gene id  \
0        uniprotkb:Q9Y287  ENSEMBL:ENSP00000367811|ENSEMBL:ENST0000064780...   
1        uniprotkb:P37840  ENSEMBL:ENSP00000338345|ENSEMBL:ENSP0000042695...   
2        uniprotkb:P0DJI8  ENSEMBL:ENSP00000348918|ENSEMBL:ENSP0000038490...   

  Interactor 1 Entrez Gene id Interactor 2 uniprot id  \
0                           -        uniprotkb:Q9Y287   
1                           -        uniprotkb:P37840   
2                           -        uniprotkb:P0DJI8   

                        Interactor 2 Ensembl gene id  \
0  ENSEMBL:ENSP00000367811|ENSEMBL:ENST0000064780...   
1  ENSEMBL:ENSP00000338345|ENSEMBL:ENSP0000042695...   
2  ENSEMBL:ENSP00000348918|ENSEMBL:ENSP0000038490...   

  Interactor 2 Entrez Gene id      Interaction type  
0                           -  physical association  
1                           -  physical association  
2                           -  physical association  


/tmp/ipykernel_35927/1739281744.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gene1 = row[0].strip("uniprotkb:")
/tmp/ipykernel_35927/1739281744.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gene2 = row[3].strip("uniprotkb:")
/tmp/ipykernel_35927/1739281744.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  reaction_id = row[7].strip("reactome:")


In [3]:
def Property(id):
    reactionname = id
    #print (reactionname)
    reactionProperty = "Unknown"
    if (reactionname in ProteinReaction.keys()):
        reactionProperty = "ProteinReaction"
    elif (reactionname in ComplexReaction.keys()):
        reactionProperty = "ComplexReaction"
    elif (reactionname in EwasReaction.keys()):
        reactionProperty = "EwasReaction"
    return reactionProperty

In [87]:
import libsbml
import glob
import re
import os
reaction_dir = "/public/ref/Msigdb/Reactome/HumanReactions"
xml_list = glob.glob(reaction_dir + "/*.sbml")
print(len(xml_list))

"""
sbml_test_file = reaction_dir + "R-HSA-156581.sbml"
reader = libsbml.SBMLReader()
document = reader.readSBML(sbml_test_file)
model = document.getModel()
for r in range(model.getNumReactions()):
        rxn = model.getReaction(r)
        for m in range(rxn.getNumModifiers()):
            mod = rxn.getModifier(m)
            print(mod.getId(), "in reaction", rxn.getId())
"""
## 依次解析内容
node_dict = {} # 存节点信息
fp = open("reactome_reaction.preedges.txt","w")
for i in range(len(xml_list)):
    pathway_ID = os.path.basename(xml_list[i]).rstrip(".sbml")
    reader = libsbml.SBMLReader()
    document = reader.readSBML(xml_list[i])
    model = document.getModel()
    for r in range(model.getNumReactions()):
        rxn = model.getReaction(r)
        reaction_id = rxn.getId()
        reaction_name = rxn.getName()
        input_nodes = []
        output_nodes = []
    #process_reactome(rxn,"Modifers")
        for m in range(rxn.getNumReactants()):
            mod = rxn.getReactant(m)
            x = re.split("_| ",mod.getId())
            input_nodes.append("R-HSA-" + x[3])
        #for e in range(rxn.getNumModifiers()):
            #mod = rxn.getModifier(e)
            #x = re.split("_| ",mod.getId())
            #input_nodes.append("R-HSA-" + x[3])
        for p in range(rxn.getNumProducts()):
            mod = rxn.getProduct(p)
            x = re.split("_| ",mod.getId())
            output_nodes.append("R-HSA-" + x[3])
        for i in iter(input_nodes):
            for j in iter(output_nodes):
                fp.write('\t'.join([i,j,reaction_id,rxn.getName(),pathway_ID])+"\n")
fp.close()
print ("start writing")
'''
fnodes =  open("reactome_reaction.nodes.txt","w")

for key,items in node_dict.items():
    include_gene = ""
    if key in PPI_dict.keys():
        include_gene = ','.join(list(set(PPI_dict[key])))
        fnodes.write(key+"\t"+items+'\t'+include_gene+"\n")
    elif key in reaction_gene_dict.keys():
        include_gene = ','.join(list(set(reaction_gene_dict[key])))
        fnodes.write(key+"\t"+items+'\t'+include_gene+"\n")
    else:
        fnodes.write(key+"\t"+items+"\n")
fnodes.close()
print ("write done")
'''

2629
start writing


'\nfnodes =  open("reactome_reaction.nodes.txt","w")\n\nfor key,items in node_dict.items():\n    include_gene = ""\n    if key in PPI_dict.keys():\n        include_gene = \',\'.join(list(set(PPI_dict[key])))\n        fnodes.write(key+"\t"+items+\'\t\'+include_gene+"\n")\n    elif key in reaction_gene_dict.keys():\n        include_gene = \',\'.join(list(set(reaction_gene_dict[key])))\n        fnodes.write(key+"\t"+items+\'\t\'+include_gene+"\n")\n    else:\n        fnodes.write(key+"\t"+items+"\n")\nfnodes.close()\nprint ("write done")\n'

In [12]:
def process_reactome(rxn,reaction):
    if (reaction == "Modifers"):
        num = rxn.getNumModifiers()
    elif (reaction == "Reactants"):
        num = rxn.getNumReactants()
    elif (reaction == "Products"):
        num = rxn.getNumProducts()
    for m in range(num):
        if (reaction == "Modifers"):
            mod = rxn.getModifier(m)
        elif (reaction == "Reactants"):
            mod = rxn.getReactant(m)
        elif (reaction == "Products"):
            mod = rxn.getProduct(m)
        x = re.split("_| ",mod.getId())
        reaction1_node = "R-HSA-" + x[1] ## Reaction
    # x[2] is connection : input/output/catalyst/positiveregulator/negativeregulator
        reaction2_node = "R-HSA-"+ x[3]                       
        print ('\t'.join([reaction1_node,reaction2_node,x[2],rxn.getId(),rxn.getName()]))
    #input_nodes.append(relation1_node + "\t" + x[2])
    #if (relation2_node != 'output'):
        #input_nodes.append(relation2_node + "\t" + x[2])
    #    print ()
    #else:
        #output_nodes.append(relation2_node)
    #return (reaction_node,complex_node,relation)

In [ ]:
import libsbml
import glob
import re
import os
reaction_dir = "/public/ref/Msigdb/Reactome/HumanReactions/"
sbml_test_file = reaction_dir + "R-HSA-392499.sbml" # contain 8956184
#sbml_test_file = reaction_dir + 'R-HSA-69002.sbml'
reader = libsbml.SBMLReader()
document = reader.readSBML(sbml_test_file)
model = document.getModel()
print (model)
input_nodes = []
output_nodes = []
fp = open("reactome_reaction.edges.txt","w")
for r in range(model.getNumReactions()):
    rxn = model.getReaction(r)
    reaction_id = rxn.getId()
    #print (reaction_id)
    input_nodes = []
    output_nodes = []
    #process_reactome(rxn,"Modifers")
    for m in range(rxn.getNumReactants()):
        mod = rxn.getReactant(m)
        x = re.split("_| ",mod.getId())
        input_nodes.append(x[3])
    #for e in range(rxn.getNumModifiers()):
       # mod = rxn.getModifier(e)
        #x = re.split("_| ",mod.getId())
        #input_nodes.append(x[3])
    for p in range(rxn.getNumProducts()):
        mod = rxn.getProduct(p)
        x = re.split("_| ",mod.getId())
        output_nodes.append(x[3])
    for i in iter(input_nodes):
        for j in iter(output_nodes):
            print ('\t'.join([i,j,reaction_id,rxn.getName()]))
    #break

In [1]:
## 处理预生成的reactome_reaction.preedges.txt，根据input - output 信息确定reaction之间的边关系
# 先记录reaction_id 对应的所有output，再遍历一遍，若input是某个reaction的output之一，则记录
import pandas as pd
preEdge = pd.read_csv("/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reactome_reaction.preedges.txt",header=None,sep='\t',index_col=False,low_memory=False)
preEdge.columns = ["input","output","reaction","reaction_name","pathway"]
print (preEdge.shape)
preEdge = preEdge.loc[:,["input","output","reaction"]].drop_duplicates(keep='first').reset_index(drop=True)
print (preEdge.shape)
reaction_output_dict ={}
preEdge['reaction'] = preEdge['reaction'].str.replace("reaction_","R-HSA-") ## 替换reaction name 由reaction_xxx -> R-HSA-xxx
print (preEdge.iloc[0:5,0:5])
print (preEdge.loc[1,"input"])
for i in range(preEdge.shape[0]): 
    reaction_output_dict.setdefault(preEdge.loc[i,"reaction"],[]).append(preEdge.loc[i,"output"])

(207075, 5)
(44544, 3)
           input         output       reaction
0  R-HSA-5227016  R-HSA-5226961  R-HSA-5226999
1  R-HSA-5216222  R-HSA-5226961  R-HSA-5226999
2    R-HSA-29926   R-HSA-109496  R-HSA-5336466
3  R-HSA-5216222  R-HSA-5226996  R-HSA-5226979
4   R-HSA-420433  R-HSA-5226996  R-HSA-5226979
R-HSA-5216222


In [12]:
## 文章里面的reaction edges 40032, preEdge反应物中的唯一关系对（ppi）为35728，文章还有其他物种

fp = open("reactome_reaction.edges.txt","w")
#fe = open("reactome_reaction.product-reactant.edges.txt","w")
record_edge = {}
#product_reactant = {}
for j in range(preEdge.shape[0]):
    input_input = preEdge.loc[j,"input"]
    input_reaction = preEdge.loc[j,"reaction"]
    for key in reaction_output_dict.keys():## key 即为output_reaction      
        if (input_input in reaction_output_dict[key]): # 一个reaction的input是另一个reaction的output
            edge1_property = Property(key)
            edge2_property = Property(input_reaction)
            #product_reactant['\t'.join(key,input_reaction)]
            record_edge['\t'.join([key,input_reaction,edge1_property,edge2_property])] = 1
            #fp.write('\t'.join([key,input_reaction])+ '\n')
        else:
            pass
    #break
#print (record_edge.shape)
for k1 in record_edge.keys():
    fp.write(k1+ '\n')
fp.close()


In [4]:
fnodes =  open("reactome_reaction.nodes.txt","w")
#for key,items in reaction_output_dict.items():
uniq_key = set(reaction_output_dict.keys())
for keys in uniq_key:
    include_gene = ""
    if keys in PPI_dict.keys():
        include_gene = ','.join(list(set(PPI_dict[keys])))
        fnodes.write(keys+"\t"+include_gene+"\n")
    elif key in reaction_gene_dict.keys():
        include_gene = ','.join(list(set(reaction_gene_dict[keys])))
        fnodes.write(keys+"\t"+include_gene+"\n")
    else:
        fnodes.write(keys+"\n")
fnodes.close()
print ("write done")

NameError: name 'PPI_dict' is not defined

In [105]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int64)
    return labels_onehot

In [108]:
import numpy as np
encode_onehot([1,2,3,4,5,6,6,7,8,8,9,0,10])

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])